In [1]:
# This script generates deep learning training data
# Author: Nikita Mishra
# run: python Deep_learning_training_data.py

# TODO: remove repeated solutions and change start and end position accordingly
import sys
import json
import pandas as pd
import numpy as np
import os
from difflib import SequenceMatcher
from bs4 import BeautifulSoup
from pathlib import Path
from importlib.machinery import SourceFileLoader
import api_client
from api_client import *
import random


#api_client = SourceFileLoader("module.name", "../api_client.py").load_module()

DATA_DIR = 'data/'
SOLUTIONS_FILE = ''.join([DATA_DIR, 'solutions_org'])
QUERIES_FILE   = ''.join([DATA_DIR, 'queries_org'])
CAUCUSES_FILE  = ''.join([DATA_DIR, 'caucuses_org'])

MAX_OUTPUT = 1000000

MAX_LENGTH = 1000
orgs_api = ApiClient('http://localhost:3002', '/v1/orgs')
def create_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
    return

def write(df, filename):
    # takes a pandas dataframe and write as json file
    df = df.to_json()
    with open(filename, 'w') as outfile:
        json.dump(df, outfile)
    print('Writing to json file {}'.format(filename))

def read(filename):
    # takes a json filename as input and loads a pandas dataframe
    with open(filename) as data_file:
        solutions = json.load(data_file)
    print('Loading from file ', filename, '...')
    return pd.read_json(solutions)

# def load(org_id, option):  
#     if option     == 'solutions':
#         file_init, api_call = SOLUTIONS_FILE, api_client.solutions_api.get
#         params    = {'org_id': org_id, 'is_canonical': 'true'}
#     elif option   == 'queries':
#         file_init, api_call = QUERIES_FILE, api_client.queries_api.get
#         params    = {'org_id': org_id}
#     elif option   == 'caucuses':
#         file_init, api_call = CAUCUSES_FILE, api_client.caucuses_api.get
#         params    ={'org_id':org_id}
#     else:
#         exit('Usage: load( org_id, <solutions, queries or caucuses>) ')
         
#     output_file = ''.join([file_init, str(org_id), '.json'])    
#     if Path(output_file).exists():
#         print('Downloaded-',option, 'file already exists!')
#         final_output = read(output_file)        
#     else:
#         print('Downloading  and saving ',option)
#         limit = 50
#         offset = 0
#         final_output = []
#         while True:
#             params['limit'], params['offset'] = limit, offset
#             single_output = api_call(params=params)
#             offset += len(single_output)
#             final_output = final_output + single_output
#             for sol in single_output:
#                 if 'content' in sol.keys():
#                     # Convert HTML tags
#                     sol['content'] = BeautifulSoup(sol['content'], 'html.parser').get_text()               
                
#             if len(final_output)%1000==0:
#                 print('Fetched %d single_output (total: %d)' % (len(single_output), len(final_output)))
#             if len(single_output) == 0 or len(final_output) >= MAX_OUTPUT:
#                 break
        
#         write(pd.DataFrame(final_output), output_file)
#     return pd.DataFrame(final_output)

def load(org_id, option):  
    if option     == 'solutions':
        file_init, api_call = SOLUTIONS_FILE, api_client.solutions_api.get
        params    = {'org_id': org_id, 'is_canonical': 'true'}
    elif option   == 'queries':
        file_init, api_call = QUERIES_FILE, api_client.queries_api.get
        params    = {'org_id': org_id}
    elif option   == 'caucuses':
        file_init, api_call = CAUCUSES_FILE, api_client.caucuses_api.get
        params    ={'org_id':org_id}
    else:
        exit('Usage: load( org_id, <solutions, queries or caucuses>) ')
         
    output_file = ''.join([file_init, str(org_id), '.json'])    
    if Path(output_file).exists():
        print('Downloaded-',option, 'file already exists!')
        final_output = read(output_file)        
    else:
        print('Downloading  and saving ',option)
        limit = 50
        offset = 0
        final_output = []
        while True:
            params['limit'], params['offset'] = limit, offset
            single_output = api_call(params=params).json()
            offset += len(single_output)
            final_output = final_output + single_output
            for sol in single_output:
                if 'content' in sol.keys():
                    # Convert HTML tags
                    sol['content'] = BeautifulSoup(sol['content'], 'html.parser').get_text()               
                
            if len(final_output)%1000==0:
                print('Fetched %d single_output (total: %d)' % (len(single_output), len(final_output)))
            if len(single_output) == 0 or len(final_output) >= MAX_OUTPUT:
                break
        
        write(pd.DataFrame(final_output), output_file)
    return pd.DataFrame(final_output)

# def query_snippet_caucus(org_id):
#     print('Running for org ->', org_id)
#     queries   = load(org_id,'queries')
#     solutions = load(org_id,'solutions')
#     caucuses  = load(org_id,'caucuses')
#     solutions = solutions.rename(columns = {'id':'solution_id','content':'solution_content'})
#     queries   = queries.rename(columns = {'id':'query_id','content':'query_content'})    
    
#     caucuses  = caucuses.loc[caucuses['expert_upvotes'] > caucuses['expert_downvotes'] ]
    
#     caucuses  = pd.merge(caucuses, queries, on='query_id', how='inner')
#     caucuses  = pd.merge(caucuses, solutions, on='solution_id', how='inner')
    
#     solutions = caucuses[['solution_content','solution_id']]
#     queries   = caucuses[['query_content','query_id']]
    
#     # get unique solutions and get there start and end position in joined passage of unique solutions
#     solutions_joined = ''.join( [row[0] for index, row in solutions.iterrows()])

#     ## Start and End of the index are the start and end positions in solutions_joined
#     for index, row in solutions.iterrows():
#         n = len(row['solution_content'])
#         if index==0:
#             Start, End = [0], [n-1]
#         else:
#             Start.append(End[-1] + 1)
#             End.append(Start[-1] + n -1 )
            
#     caucuses.insert(1, 'start', Start)
#     caucuses.insert(1, 'end', End)    
#     print('Summary: caucus_size = ',len(caucuses),', solution_size',len(solutions),',query_size',len(queries),'\n done!')    
#     return solutions_joined, caucuses[['org_id','query_id', 'solution_id','id','query_content', 'solution_content']]

def query_snippet_caucus(org_id):
    print('Running for org ->', org_id)
    
    output_file = ''.join([CAUCUSES_FILE,'_filtered', str(org_id), '.json'])
    if not Path(output_file).exists():
        
        queries   = load(org_id,'queries')
        solutions = load(org_id,'solutions')
        caucuses  = load(org_id,'caucuses')
        solutions = solutions.rename(columns = {'id':'solution_id','content':'solution_content'})
        queries   = queries.rename(columns = {'id':'query_id','content':'query_content'})    

        caucuses  = pd.merge(caucuses, queries, on='query_id', how='inner')
        caucuses  = pd.merge(caucuses, solutions, on='solution_id', how='inner')
        caucuses  = caucuses[['org_id','query_id', 'solution_id','id','query_content', 'solution_content', 'expert_upvotes','expert_downvotes' ]]

        write(caucuses, output_file)
        print('Summary: caucus_size = ',len(caucuses),', solution_size',len(solutions),',query_size',len(queries),'\n done!')    
    
    else:
        caucuses = read(output_file)
        
    return caucuses
def query_solution_pairs(queries, solutions):
    
    output_file = ''.join([QUERIES_FILE,'_filtered', str(org_id), '.json'])
    if not Path(output_file).exists():
        queries_filtered = []
        for index, row in queries.iterrows():
            try:
                if 'source_id' in row['sources'][0] and len(row['content']) < MAX_LENGTH:
                    queries_filtered.append([row['sources'][0]['source_id'], row['id'], remove_special_char(row['content']) ])
            except:
                print(index,row)
                exit(1)
        queries_filtered   = pd.DataFrame( queries_filtered)
        queries_filtered.columns = [ 'resource_id','query_id','query_content']
        queries_filtered = queries_filtered.drop_duplicates(subset='resource_id')
        write(queries_filtered, output_file)
    else:
        queries_filtered = read(output_file)
    output_file = ''.join([SOLUTIONS_FILE,'_filtered', str(org_id), '.json'])    
    if not Path(output_file).exists():
        solutions_filtered = [[row['resource']['id'], row['id'],  remove_special_char(row['content'])] 
                    for index, row in solutions.iterrows() if len(row['content']) < MAX_LENGTH ]
        solutions_filtered = pd.DataFrame(solutions_filtered)
        solutions_filtered.columns = [ 'resource_id','solution_id','solution_content']
        solutions_filtered = solutions_filtered.drop_duplicates(subset='resource_id')
        write(solutions_filtered, output_file)
    else:
        solutions_filtered = read(output_file)        
    qs_pairs  = queries_filtered.merge(solutions_filtered, on='resource_id', how='inner')
    return qs_pairs


def split(df,p_train,p_test,p_dev):
    df = df.sample(frac=1, random_state=9001).reset_index(drop=True)
    n = p_train+p_test+p_dev # just to make sure we have the split normalize
    p_train,p_test,p_dev = p_train/n, p_test/n, p_dev/n    
    train, validate, test = np.split(df.sample(frac=1), [int(p_train*len(df)), int((p_train+p_dev)*len(df))])
    
    dir_split_sample = ''.join([DATA_DIR, 'split_',str(org_id)])
    create_dir(dir_split_sample )
    
    train_dir = ''.join([dir_split_sample ,'/train'])
    test_dir  = ''.join([dir_split_sample ,'/test'])
    dev_dir   = ''.join([dir_split_sample ,'/dev'])
    
    create_dir(train_dir)
    create_dir(test_dir)
    create_dir(dev_dir)

    train['query_content'].to_csv(      ''.join([train_dir,'/sources.txt']), index=False,header=False)
    train['solution_content'].to_csv(   ''.join([train_dir,'/targets.txt']), index=False,header=False)

    validate['query_content'].to_csv(   ''.join([dev_dir,  '/sources.txt']), index=False,header=False)
    validate['solution_content'].to_csv(''.join([dev_dir,  '/targets.txt']), index=False,header=False)

    test['query_content'].to_csv(       ''.join([test_dir, '/sources.txt']), index=False,header=False)
    test['solution_content'].to_csv(    ''.join([test_dir, '/targets.txt']), index=False,header=False)
    return (train, validate, test)

def split_sample(df,k,p_train,p_test,p_dev):
    df = df.sample(frac=1, random_state=9001).reset_index(drop=True)
    df = df.iloc[:k]
    n = p_train+p_test+p_dev # just to make sure we have the split normalize
    p_train,p_test,p_dev = p_train/n, p_test/n, p_dev/n    
    train, validate, test = np.split(df.sample(frac=1), [int(p_train*len(df)), int((p_train+p_dev)*len(df))])
    
    dir_split_sample = ''.join([DATA_DIR, 'split_sample_',str(org_id)])
    create_dir(dir_split_sample )
    
    train_dir = ''.join([dir_split_sample ,'/train'])
    test_dir  = ''.join([dir_split_sample ,'/test'])
    dev_dir   = ''.join([dir_split_sample ,'/dev'])
    
    create_dir(train_dir)
    create_dir(test_dir)
    create_dir(dev_dir)

    train['query_content'].to_csv(      ''.join([train_dir,'/sources.txt']), index=False,header=False)
    train['solution_content'].to_csv(   ''.join([train_dir,'/targets.txt']), index=False,header=False)

    validate['query_content'].to_csv(   ''.join([dev_dir,  '/sources.txt']), index=False,header=False)
    validate['solution_content'].to_csv(''.join([dev_dir,  '/targets.txt']), index=False,header=False)

    test['query_content'].to_csv(       ''.join([test_dir, '/sources.txt']), index=False,header=False)
    test['solution_content'].to_csv(    ''.join([test_dir, '/targets.txt']), index=False,header=False)
    return (train, validate, test)
# def remove_special_char(string):
#     #return ''.join(e for e in string if e.isalnum())
#     string = string.replace('"', '')
#     string = string.replace('\'', '')
#     string = string.replace('\r', '')
    
#     return string.replace('\n', '.')

def remove_special_char(string):
    strings_to_remove = ['Upwork Customer Support | . Blog: Building a Mobile App? A Prototype Saved Our Product and link to https://www.upwork.com/blog/2014/08/building-mobile-app-prototype-saved-product/.'  
                         ,'\"','\'','\r','Hello', 'UNK', 'Thank you for contacting Upwork', 'Support &#124', 
                         'support.upwork.com', 'From the Upwork', 'Thanks for contacting Upwork','Upwork.com.',
                         'Upwork Support Team','High Value Support',
                         'Upwork Marketplace Quality Team'
                        ]
    
    for item in strings_to_remove:
        string = string.replace(item, '')
    
    return string.replace('\n', '.')
# def tokenize_and_copy(org_name):
#     !mkdir /Users/nikita/nmt_data/$org_name
#     !cp -r data/$org_name/* /Users/nikita/nmt_data/$org_name
#     !bash /Users/nikita/Desktop/seq2seq/bin/data/tokenize.sh $org_name
#     !scp -r /Users/nikita/nmt_data/$org_name deep_machine:/tmp/nmt_tutorial

def tokenize_and_copy(org_id, org_name,typee):
    split_dir  = ''.join([DATA_DIR, 'split_',typee,'_',str(org_id)])
    nmt        = ''.join([ org_name,'_',typee])

    !mkdir /Users/nikita/nmt_data/$nmt
    !cp -r $split_dir/* /Users/nikita/nmt_data/$nmt
    !bash /Users/nikita/Desktop/seq2seq/bin/data/tokenize.sh $nmt
    !scp -r /Users/nikita/nmt_data/$nmt deep_machine:/tmp/nmt_tutorial

def save_snippet(df):
    
    dir_split_sample = ''.join([DATA_DIR, 'split_snippet_',str(org_id)])
    create_dir(dir_split_sample )
    
    train_dir = ''.join([dir_split_sample ,'/train'])
    
    create_dir(train_dir)
    train = df
    train['query_content'].to_csv(      ''.join([train_dir,'/sources.txt']), index=False,header=False)
    train['solution_content'].to_csv(   ''.join([train_dir,'/targets.txt']), index=False,header=False)
    train.to_csv(   ''.join([train_dir,'/df.txt']), index=False,header=False)
    return 
    
# if __name__ == '__main__':
#     list_of_orgs = pd.read_csv('/tmp/list_of_active_orgs', delimiter=' ', header=None).iloc[0]
#     for org_id in list_of_orgs:
#         solutions_joined, caucuses = run(org_id)

In [ ]:
create_dir(DATA_DIR)
           
#org_id,org_name    = 7, 'alexa'
org_id, org_name,typee   = 1, 'Upwork','snippet'

        

#org_name  = orgs_api.get(path="/%s" % org_id)['name']
queries   = load(org_id,'queries')
solutions = load(org_id,'solutions')
qs_pairs  = query_solution_pairs(queries, solutions)

(train,validate,test) = split(qs_pairs,0.9,0.1,0.1)
(train_sample,validate_sample,test_sample) = split_sample(qs_pairs,1000,0.9,0.1,0.1)

tokenize_and_copy(org_id, org_name,'')
tokenize_and_copy(org_id, org_name,'sample')

# tokenize_and_copy(''.join(org_name))
# tokenize_and_copy(''.join([org_name,'_sample']))



In [5]:
org_id, org_name,typee   = 1, 'Upwork','snippet'
caucuses = query_snippet_caucus(1)
save_snippet(caucuses) 
tokenize_and_copy(org_id, org_name,typee)


Running for org -> 1
Loading from file  data/caucuses_org_filtered1.json ...
mkdir: /Users/nikita/nmt_data/Upwork_snippet: File exists
Writing to /Users/nikita/nmt_data/Upwork_snippet. To change this, set the OUTPUT_DIR environment variable.
Tokenizing /Users/nikita/nmt_data/Upwork_snippet/train/df.txt...
Tokenizing /Users/nikita/nmt_data/Upwork_snippet/train/sources.txt...
Tokenizing /Users/nikita/nmt_data/Upwork_snippet/train/targets.txt...
Wrote /Users/nikita/nmt_data/Upwork_snippet/train/vocab.sources.tok
Wrote /Users/nikita/nmt_data/Upwork_snippet/train/vocab.targets.tok
All done.
df.tok                                        100% 4984KB  24.5MB/s   00:00    
df.txt                                        100% 4668KB  23.9MB/s   00:00    
sources.tok                                   100% 1417KB  23.4MB/s   00:00    
sources.txt                                   100% 1310KB  23.1MB/s   00:00    
targets.tok                                   100% 2703KB  19.8MB/s   00:00    
targets

In [ ]:
q_length = []
s_length = []
for index in range(len(qs_pairs)):
    q_length.append(len(qs_pairs.iloc[index]['query_content']))
    s_length.append(len(qs_pairs.iloc[index]['solution_content']))
q_length = sorted(q_length)
s_length = sorted(s_length)  
print(np.median(q_length), np.median(s_length), max(q_length), max(s_length), min(q_length), min(s_length))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 
plt.plot(q_length)

In [ ]:
%matplotlib inline 
plt.plot(s_length)

In [2]:
# Redo negative examples to have similar length distribution

create_dir(DATA_DIR)
           
#org_id,org_name    = 7, 'alexa'
org_id, org_name,typee   = 1, 'Upwork','snippet'

        

#org_name  = orgs_api.get(path="/%s" % org_id)['name']
queries   = load(org_id,'queries')
solutions = load(org_id,'solutions')

Downloaded- queries file already exists!
Loading from file  data/queries_org1.json ...
Downloaded- solutions file already exists!
Loading from file  data/solutions_org1.json ...
